# Neuroidal Model Simulation v1.2
#### Patrick Perrine

Code for:

Perrine, P. R. (2023). Neural Tabula Rasa: Foundations for Realistic Memories and Learning. Master's thesis. California Polytechnic State University, San Luis Obispo.

All algorithms implemented here are originally given by:

Valiant, L. G. (2005). Memorization and Association on a Realistic Neural Model. *Neural Computation, 17*, 527–555. https://doi.org/10.1162/0899766053019890

Many thanks to Mugizi Rwebangira and Chandradeep Chowdhury for their constant support.

## Introduction
We offer the following simulation as an accurate interpretation of the Neuroidal model, as described specifically by Valiant (2005). We place an emphasis on simulating the model to study the behavior of the one-step, shared memory representation for use in unsupervised memorization via the JOIN algorithm. We offer a general implementation framework that should easily extend to the other variants of the model. We choose the Python library *graph-tool* to serve as the basis for storing and performing operations on our random graph model. We opt for this particular library due to its reported performance, code readability, and implementations for visualizing very large, dense graphs (which we leave for future work).

We omit an implementation of the LINK procedure, as we do not measure this model based on its ability to recall information. We are primarily concerned with how much information can be stored using JOIN before an intolerable amount of interference occurs between new and pre-existing memories.

### Install the *graph-tool* library (https://graph-tool.skewed.de/)
The following two code blocks are subject to change based on the library's current hosting location, so referring to the current information from its website may be required. If one needs to run this code without Jupyter: Ignore the following two blocks and please refer to the library's current documentation for proper installation instructions for the desired setup.

In [1]:
%%capture
!echo "deb http://downloads.skewed.de/apt jammy main" >> /etc/apt/sources.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-matplotlib python3-cairo

In [2]:
# Run this block if you are on Google Colab, otherwise skip
%%capture
!apt purge python3-cairo
!apt install libcairo2-dev pkg-config python3-dev
!pip install --force-reinstall pycairo
!pip install zstandard

### Import other Required Libraries and Seed Random Number Generator

In [3]:
%%capture
import itertools
import numpy as np
from numpy.random import *
import graph_tool.all as gt

In [4]:
global rng
rng = np.random.default_rng(seed=42)

### Define the Neuroidal Model's Empirical Properties
The following configuration for simulating the one-step, shared representation model, with the intention to adhere to results given in Tables 3 and 4 of Valiant (2005). The first five parameters are as described in the referenced paper. The $r_{approx}$ parameter requires prior determination as pursuant to the relations defined in the paper, and is for generating initial memories in a fixed size to be used in JOIN.

We also implicitly declare all empirical properties as global variables for convenience.

In [5]:
n = 100000
d = 512
k = 32
t = 1
k_adj = 1.9

r_approx = 5170 # From Table 3 of Valiant (2005)

#### Calculate Edge Existence Probability $p$
In the analysis of Valiant (2005) where $n \ge 10^5$, we have $p = \frac{d}{n}$.

However for lower values of $n$, we find the more general $p = \frac{d}{n-1}$ to be appropriate.

In [6]:
if n >= 10^5:
    p = d / n
else:
    p = d / (n - 1)

## Generate the Graph Model
Here we intstantiate a graph model equivalent to the Erdős-Rényi $G=(n,p)$ structure.

### Initialize an Empty Graph

In [7]:
global g
g = gt.Graph(directed=True)

### Populate our Graph with $n$ Neurons

In [8]:
%%capture
g.add_vertex(n)

### Populate our Graph with Random Synapses

In [9]:
def add_gnp_edges(): # Naive implementation
    all_edges = itertools.permutations(range(n), 2)
    for e in all_edges:
        if rng.random() < p:
            g.add_edge(*e)

The following function provides a speed-optimized (yet less readable and more memory intensive) implementation of adding a list of random edges. This version is more reminiscent of the $G=(n,M)$ model, but we determine the number of edges $M$ using Bernoulli trials.

In [10]:
def add_fast_gnp_edges():
    num_edges = rng.binomial(n*(n-1)/2, p)
    sources = rng.integers(0, n, num_edges*2)
    targets = rng.integers(0, n, num_edges*2)
    mask = sources != targets # removes self-loops
    g.add_edge_list(np.column_stack((sources[mask], targets[mask])))

In [11]:
#add_gnp_edges() # Space-Efficient
add_fast_gnp_edges() # Time-Efficient

We now have a proper $G=(n,p)$ graph to use for our Neuroidal model.

### Initialize the Mode of all Neurons and Synapses
The following two blocks imbue the global properties, or the "mode" $s$, of each node and edge to the graph.

For each neuron $i$, its mode $s_i$ contains the following properties:
* $T_i$ specifies the threshold gate value of a given node, commonly set to $1$, or as high as $7$.
* $q_i$ stores the state of a given node, initialized as $1$ and set to $2$ if a node is found to be a candidate for memory $C$ during JOIN.
  * During *two-step* procedures, this state can also be set to $3$.
  * This can be interpreted as the state $q_i$ belonging to a set of finite states $Q = \{1,2,3\}$.
  * Such states can be interpreted as representative of how a given neuron is considered "allocated" in memory or otherwise.
* $f_i$ is reinterpreted as a Boolean to show whether a give mode is firing (**True**) or not firing (**False**) at the current time step.

We omit the storing of the incoming weight sum of a node $w_{i}$ here, as it will be calculated in the weight summation functions as described later.

In [12]:
mode_T = g.new_vertex_property("int")
mode_q = g.new_vertex_property("int")
mode_f = g.new_vertex_property("bool")

mode_T.a = t
mode_q.a = 1
mode_f.a = False

For each synapse $(j,i)$, its mode $s_{ji}$ contains the following:
* $w_{ji}$ specifies the weight of the edge from node $j$ to node $i$.
  * For *disjoint* representations, this would be initialized to be $\frac{t}{k}$.
  * However, for *shared* representations, we initialize the weight to be $\frac{t}{k \cdot {k_{adj}}}$.
* $qq_{ji}$ stores the state of a given edge, set to $1$ and is not updated for one-step procedures.
  * For two-step operations, we would have $qq_{ji} \in Q = \{1,2\}$.

In [13]:
mode_w = g.new_edge_property("double")
mode_qq = g.new_vertex_property("int")

mode_w.a = t / (k_adj * k)
mode_qq.a = 1

## Vicinal Algorithms for the JOIN Algorithm

### Weight Summation Functions
The following two blocks calculate and return
$$ w_i = \sum_{(j,i)\in E}{} f_{j} \cdot w_{ji} $$
given a neuron $i$.

In [14]:
def sum_weights(v_i): # Naive implementation
    w_i = 0
    for e_ji in g.iter_in_edges(v_i):
        if mode_f[e_ji[0]] == True:
            w_i += mode_w[e_ji]
    return w_i

This method below is equivalent to the function above, but leverages speed optimizations offered by graph-tool and NumPy to negate the costly, explicit loop.

In [15]:
def fast_sum_weights(v_i):
    W = g.get_in_edges(v_i, [mode_w])[:,2]
    F = np.array(g.get_in_neighbors(v_i, [mode_f])[:,1], dtype=bool)
    return W[F].sum()

### Neuron and Synapse Mode Updates
These two blocks, along with the "mode" structure in general, are mostly an effort to remain aesthetically similar the original formulations in Valiant (2005) and the earlier work *Circuits of the Mind*. Therefore, these specific functions may be later disregarded in favor of more efficient procedures given our implementation strategy here.

$\delta\left(s_{i}, w_{i}\right) = s_{i}'$.

In [16]:
def _delta(v_i, w_i):
    if w_i > mode_T[v_i]:
        mode_q[v_i] = 2
        mode_f[v_i] = True

$\lambda\left(s_{i}, w_{i}, s_{ji}, f_{j}\right) = s_{ji}'$.

In [17]:
def _lambda(v_i, w_i, e_ji, f_j):
    if f_j == 1:
        mode_qq[e_ji] = 2

### Overall Graph Update
Note that traversal of each node to calculate the sum of its incoming weights results in one of our most costly procedures in this simulation.

We attribute this partly because the Neuroidal model was conceived as a *distributed system* with neurons and synapses acting as their own *agents*. Here they are expressed simply as objects with their attributes being manipulated by our algorithms. Hence, we are performing operations sequentially for each node in the graph, and therefore acknowledge that this overall process is not quite exact to the original formulation of the model.

After all updates, we reset the states and firing modes of each neuron/synapse, indicating that the *time step* has been completed.

#### Shortcut for Determining Memory $C$
For convenience, we check for eligible $C$ nodes during the weight sums for each neuron of the graph. We also allow the skipping of updating the edges of the graph for our use of JOIN in the simulation, as edge updates are not needed in our case.

Also, we would normally allow a neuron to fire when its threshold is surpassed by its incoming edge weights (known as a threshold transition). However, we reverse such firing here due to it affecting the search for memory $C$ when other nodes will later be checked, and threshold transitions have no other immediate use in our specific simulation.

In [18]:
def update_graph(update_edges=False):
    C = []
    for v_i in g.iter_vertices():
        w_i = fast_sum_weights(v_i)
        _delta(v_i, w_i)
        if mode_q[v_i] == 2:
            C.append(v_i)
            mode_f[v_i] = False
        if update_edges:
            for e_ji in g.iter_in_edges(v_i):
                f_j = mode_f[e_ji[0]]
                _lambda(v_i, w_i, e_ji, f_j)
    mode_q.a = 1
    mode_qq.a = 1
    mode_f.a = False
    return C

## The JOIN Algorithm
This implements the *one-step* variant of JOIN for *shared* representations as
defined in Valiant (2005), explained in further detail here:

1. Fire all neurons in given memories $A$ and $B$ "simultaneously."
2. Update the graph to determine any threshold transitions.
3. Create memory $C$ from neurons that transitioned.

We reiterate that all parts of the process are intended to encompass one time step, but this notion is discretized in our formulation here.

In [19]:
def JOIN_one_step_shared(A, B):
    for v in A + B:
        mode_f[v] = True
    return update_graph(update_edges=False)

## Interference Check Function
Here we check every memory that is not $A$ or $B$, referred to as $D$ memories, to calculate how many interfering nodes exist between each given $D$ memory and $C$. If more than $50\%$ of a $D$ memory is found to interfere with $C$, then an instance of interference has been found. We would then increment the sum of occurrences by $2$, as we consider interference to be bidirectional to both incidental memories.

In [20]:
def interference_check(S, A_i, B_i, C):
    sum = 0
    for D_i in range(len(S)):
        if D_i != A_i and D_i != B_i:
            D = S[D_i]
            if len(set(C) & set(D)) > (len(D) / 2):
                sum += 2
    return sum

## Our Simulation

### Print Functions for Simulation Updates
These three blocks are simply for providing feedback for whenever: a batch of ongoing JOIN operations completes, the model's capacity is reached, or all memory connectives have been memorized.

The interference metric reports are mostly useful for when $n < 10^5$, where the overall system is generally much less stable.

In [21]:
def print_join_update(S_length, L, H, H_if, total_if, m_len, m_total):
    print("Current Total Memories:", S_length)
    print("Batch Interference Rate:", round(H_if/H, 4))
    print("Batch Average Memory Size:", int(m_len/H))
    print("Running Average Interf. Rate:", round(total_if/S_length, 4))
    print("Running Average Memory Size:", int(m_total/(S_length-L)),"\n\n")

In [22]:
def print_halt_msg(S_length, L, F, total_if, m_total):
    F_p = int(F*100)
    r_obs = int(m_total/(S_length-L))
    r_error = round(((r_approx - r_obs) / r_obs) * 100, 2)
    print("-- End of Simulation (Halted) --\n")
    print("Given: n=", n, "d=", d, "k=", k, "k_adj=", k_adj, "r_approx=",
           r_approx, "START_MEM=", L)
    print("we halted Memory Formation at", F_p, "% Total Interference.\n")
    print("Total Average Interference Rate:", round(total_if/S_length, 4),"\n")
    print("Empirical Memory Size:", int(r_obs))
    print("Approximation Error of r:", r_error, "%\n")
    print("Capacity:", L, "Initial Memories +" ,S_length-L,"JOIN Memories.")

In [23]:
def print_memorized_msg(S_length, L, m_total):
    r_obs = int(m_total/(S_length-L))
    r_error = round(((r_approx - r_obs) / r_obs) * 100, 2)
    print("-- End of Simulation (Completed) --\n")
    print("Given: n=", n, "d=", d, "k=", k, "k_adj=", k_adj, "r_approx=",
           r_approx, "START_MEM=", L)
    print("we memorized all possible combinations of", L, "memories.\n")
    print("Empirical Memory Size:", int(r_obs))
    print("Approximation Error of r:", r_error, "%\n")
    print("Contains:", L, "Initial Memories +" ,S_length-L,"JOIN Memories.")

### Call JOIN until Capacity is reached or all connectives are Memorized

#### Simulation Parameters
* The $L$ parameter specifies how many initial memories to grant the model before calling JOIN.
  * Each initial memory will be of fixed size $r_{approx}$ and will consist of randomly chosen neurons with replacement.
* $S$ is our "memory bank" in which we will initially append $L\choose{2}$ combinations of initial memories to, and continue to add memories and check interference as a result of JOIN.
* The $F$ parameter determines the level of interference tolerance to be checked at each iteration of JOIN.
* (Optional) The $H$ parameter determines how many memories to JOIN in one batch of simulation.
  * This is mostly used to observe the model perform at an either fine or course-grain level, as determined by the user.
* All other variables used are primarily for update reporting.

Our simulation will attempt to JOIN all possible $L\choose{2}$ pairs of initial memories until the model's capacity is reached. If $L\choose{2}$ is found to be less than our capacity, then the simulation will terminate and have found the model to have memorized all possible connectives.

In [24]:
def JOIN_one_step_shared_simulation(L, F, H=1, verbose=True):
    m = 0
    H_if = 0
    m_len = 0
    m_total = 0
    total_if = 0

    print("-- Start of Simulation --\n")

    init_pairs = itertools.combinations(range(L), 2)
    S = [rng.choice(np.arange(0,n-1), size=r_approx)
         for _ in range(L)]

    for A_i,B_i in init_pairs:
        A = list(S[A_i])
        B = list(S[B_i])
        C = JOIN_one_step_shared(A, B)
        C_if = interference_check(S, A_i, B_i, C)

        m += 1
        S.append(C)
        m_len += len(C)
        m_total += len(C)

        if m % H == 0:
            if verbose:
                print_join_update(len(S), L, H, H_if, total_if, m_len, m_total)
            H_if = 0
            m_len = 0

        if C_if > 0:
            H_if += C_if
            total_if += C_if
            if total_if/len(S) > F:
              print_halt_msg(len(S), L, F, H, H_if, total_if)
              return S

    print_memorized_msg(len(S), L, m_total)
    return S

### Generate our Memory Bank and Calculate the Empirical Memory Size
Given the default parameters, we observe that there is no notable amount of interference (less than $0.3\%$) for the given amount of memories disjuncted. We are then able to verify the precision in our choice of $r_{approx}$ empirically given the performance of the simulation. We also later observe that the observed capacity of the shared model is clearly greater than the analytical capacity for a disjoint representation.

In [25]:
memory_bank = JOIN_one_step_shared_simulation(L=10,
                                              F=.003,
                                              H=10, verbose=True)

-- Start of Simulation --

Current Total Memories: 20
Batch Interference Rate: 0.0
Batch Average Memory Size: 5204
Running Average Interf. Rate: 0.0
Running Average Memory Size: 5204 


Current Total Memories: 30
Batch Interference Rate: 0.0
Batch Average Memory Size: 5162
Running Average Interf. Rate: 0.0
Running Average Memory Size: 5183 


Current Total Memories: 40
Batch Interference Rate: 0.0
Batch Average Memory Size: 5215
Running Average Interf. Rate: 0.0
Running Average Memory Size: 5194 


Current Total Memories: 50
Batch Interference Rate: 0.0
Batch Average Memory Size: 5185
Running Average Interf. Rate: 0.0
Running Average Memory Size: 5192 


-- End of Simulation (Completed) --

Given: n= 100000 d= 512 k= 32 k_adj= 1.9 r_approx= 5170 START_MEM= 10
we memorized all possible combinations of 10 memories.

Empirical Memory Size: 5188
Approximation Error of r: -0.35 %

Contains: 10 Initial Memories + 45 JOIN Memories.


#### Compare Results with the Capacity for the *Disjoint* Representation Model
Capacity is measured here by using a result from Table 1 of Valiant (2005) from a model with identical parameters, aside from the $k_{adj}$ value which is not applicable to the disjoint representation.

In [26]:
print("Analytical capacity of disjoint representation:", int(100000 / 5420))

Analytical capacity of disjoint representation: 18
